In [1]:
from ipywidgets import interact
import ipywidgets as widgets
import time
import warnings
import pandas as pd
warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
import numpy as np
import plotly.graph_objects as go

np.seterr(divide='ignore', invalid='ignore')

def plot(fig,x,y,name):
    fig.add_trace(go.Scatter(x=x,y=y,mode='lines',name=name))


SignalGeneration

In [3]:
#BoxFunction
def box(x,a=2,b=5):
    y=np.zeros(len(x))
    y[np.abs(x)<a]=b
    return y
def saw(x,a=2,b=5):
    y=np.zeros(len(x))
    y[np.abs(x)<a]=-b*x[np.abs(x)<a]
    return y
def exp(x,a=2,b=2,c=-1):
    y=np.zeros(len(x))
    y[np.abs(x)<a]=c+np.e**(-b*x[[np.abs(x)<a]])
    return y
def sinu(x,a=2,b=2,c=-1):
    y=np.zeros(len(x))
    y[np.abs(x)<a]=c*np.sin(b*x[np.abs(x)<a])
    return y
def gaus(x,a=2,b=2):
    y=np.zeros(len(x))
    y[np.abs(x)<a]=np.e**(-a*(x[np.abs(x)<a]**(2*b)))
    return np.nan_to_num(y)
def BoxFourierAnalytical(a,b,N):
    k=np.linspace(int((-N/2)),int((N/2)),N+1)
    fkhat=np.zeros(len(k))
    fkhat[k==0]=(2*a*b)
    fkhat[k!=0]=((2*b*np.sin(a*k[k!=0]))/k[k!=0])
    return fkhat/(2*np.pi)
def SawFourierAnalytical(a,b,N):
    k=np.linspace(int((-N/2)),int((N/2)),N+1)
    fkhat=np.zeros(len(k)).astype(complex)
    fkhat[k==0]=0
    fkhat[k!=0]=(2*1j*b*(np.sin(a*k[k!=0])-a*k[k!=0]*np.cos(a*k[k!=0])))/(k[k!=0]**2)
    return fkhat/(2*np.pi)
def ExpFourierAnalytical(a,b,c,N):
    k=np.linspace(int((-N/2)),int((N/2)),N+1).astype(complex)
    fkhat=np.zeros(len(k)).astype(complex)
    fkhat[k==0]=(2*((a*b*c)+np.sinh(a*b)))/b
    fkhat[k!=0]=((2*c*np.sin(a*k[k!=0]))/k[k!=0])+((2*np.sinh(a*(b+(1j*k[k!=0]))))/(b+(1j*k[k!=0])))
    return fkhat/(2*np.pi)
def SinuFourierAnalytical(a,b,c,N):
    k=np.linspace(int((-N/2)),int((N/2)),N+1).astype(complex)
    fkhat=np.zeros(len(k)).astype(complex)
    fkhat[k==b]=((1j*c*np.sin(2*a*b))/(2*b))-(1j*a*c)
    fkhat[k==-b]=(1/2)*1j*c*((2*a)-((np.sin(2*a*b))/b))
    fkhat[k!=b]=(2*1j*c*((b*np.cos(a*b)*np.sin(a*k[k!=b]))-(k[k!=b]*np.sin(a*b)*np.cos(a*k[k!=b]))))/((b**2)-(k[k!=b]**2))
    return fkhat/(2*np.pi)
def FourierSeries(cn,X,N):
    fx=[]
    '''
    Cn=np.zeros(len(cn))
    Cn[0]=cn[int(N/2)]
    Cn[1:int(N/2)-1]=cn[1:int(N/2)-1]
    Cn[int(N/2)+1:N]=cn[int(N/2)+1:N]
    fx=np.fft.ifft(Cn)
    '''
    
    for x in X:
        result=0
        for i in range(int((-N/2)),int((N/2))):
            result=result+cn[int(i+(N/2))]*(np.e**(1j*i*x)) #Calculate Fourier Series approximation using formula
        fx.append(result.real)
    return fx

x=np.linspace(-np.pi,np.pi,100)


In [4]:
def dft(N,M=40):
    #w=np.e**((2*np.pi*-1j)/N)
    power=np.zeros((M,N),dtype=complex)
    xj = np.linspace(-np.pi, np.pi, N)
    for i in range(M):
        for j in range(N):
            #power[i][j]=((-N/2)+i)*j
            power[i][j] = -1j*(i-M/2)*xj[j]
    #DFT=(w**power)/N
    DFT = np.e**power
    return DFT
def box(x,a=2,b=5,normalized=True,noise=False,noiseParameter=0.1,fourier=False,N=40):
    if fourier:
        k=np.linspace(int((-N/2)),int((N/2)),N+1)
        fkhat=np.zeros(len(k))
        fkhat[k==0]=(2*a*b)
        fkhat[k!=0]=((2*b*np.sin(a*k[k!=0]))/k[k!=0])
        return fkhat/(2*np.pi)
    y=np.zeros(len(x))
    y[np.abs(x)<a]=b
    if noise:
        return y+np.random.normal(0,noiseParameter,len(x))
    return y
def saw(x,a=2,b=5,noise=False,noiseParameter=0.1,fourier=False,N=40):
    if fourier:
        k=np.linspace(int((-N/2)),int((N/2)),N+1)
        fkhat=np.zeros(len(k)).astype(complex)
        fkhat[k==0]=0
        fkhat[k!=0]=(2*1j*b*(np.sin(a*k[k!=0])-a*k[k!=0]*np.cos(a*k[k!=0])))/(k[k!=0]**2)
        return fkhat/(2*np.pi)
    y=np.zeros(len(x))
    y[np.abs(x)<a]=-b*x[np.abs(x)<a]
    if noise:
        return y + np.random.normal(0,noiseParameter,len(x))
    return y
def exp(x,a=2,b=2,c=-1,noise=False,noiseParameter=0.1,fourier=False,N=40):
    if fourier:
        k=np.linspace(int((-N/2)),int((N/2)),N+1).astype(complex)
        fkhat=np.zeros(len(k)).astype(complex)
        fkhat[k==0]=(2*((a*b*c)+np.sinh(a*b)))/b
        fkhat[k!=0]=((2*c*np.sin(a*k[k!=0]))/k[k!=0])+((2*np.sinh(a*(b+(1j*k[k!=0]))))/(b+(1j*k[k!=0])))
        return fkhat/(2*np.pi)
    y=np.zeros(len(x))
    y[np.abs(x)<a]=c+np.e**(-b*x[[np.abs(x)<a]])
    if noise:
        return y + np.random.normal(0,noiseParameter,len(x))
    return y
def sinu(x,a=2,b=2,c=-1,noise=False,noiseParameter=0.1,fourier=False,N=40):
    if fourier:
        k=np.linspace(int((-N/2)),int((N/2)),N+1).astype(complex)
        fkhat=np.zeros(len(k)).astype(complex)
        fkhat[k==b]=((1j*c*np.sin(2*a*b))/(2*b))-(1j*a*c)
        fkhat[k==-b]=(1/2)*1j*c*((2*a)-((np.sin(2*a*b))/b))
        fkhat[k!=b]=(2*1j*c*((b*np.cos(a*b)*np.sin(a*k[k!=b]))-(k[k!=b]*np.sin(a*b)*np.cos(a*k[k!=b]))))/((b**2)-(k[k!=b]**2))
        return fkhat/(2*np.pi)
    y=np.zeros(len(x))
    y[np.abs(x)<a]=c*np.sin(b*x[np.abs(x)<a])
    if noise:
        return y + np.random.normal(0,noiseParameter,len(x))
    return y
def gaus(x,a=2,b=2,noise=False,noiseParameter=0.1,fourier=False,N=40,M=40):
    if fourier:
        fkhat=((np.dot(dft(N,M),gaus(x,a,b)))/N)
        return fkhat
    y=np.zeros(len(x))
    y[np.abs(x)<a]=np.e**(-a*(x[np.abs(x)<a]**(2*b)))
    y=np.nan_to_num(y)
    if noise:
        return y + np.random.normal(0,noiseParameter,len(x))
    return y

# Visualization of signals

In [5]:
N=100
x=np.linspace(-np.pi,np.pi,N)
fig=go.Figure()
fig.add_trace(go.Scatter(x=x,y=box(x,0.1,0.1),mode='lines',name='Box'))
fig.update_xaxes(range=[-np.pi,np.pi])
fig.update_yaxes(range=[-1,1])


k=np.linspace(0,N,N)
#print(fx)
a=0
b=0


@interact(a=(0.1,3,0.1),b=(-1.1, 1.1, 0.1))
def update(FourierRepresentation=False,a=0.1,b=0.1):
    with fig.batch_update():
        fig.data[0].y=box(x,a,b)
        if(FourierRepresentation):
            if(len(fig.data)==1):
                #cn=BoxFourierAnalytical(a,b,N)
                cn=box(x,a,b,fourier=True,N=100)
                fx=FourierSeries(cn,x,N)
                plot(fig,x,fx,'FourierRepresentation')
            else:
                cn=box(x,a,b,fourier=True,N=100)
                fx=FourierSeries(cn,x,N)
                fig.data[1].y=fx
        else:
            fig.data = [fig.data[0]]
        fig.show()



interactive(children=(Checkbox(value=False, description='FourierRepresentation'), FloatSlider(value=0.1, descr…

## 2. Sawtooth

In [6]:
fig=go.Figure()

fig.add_trace(go.Scatter(x=x,y=saw(x,0.1,0.1),mode='lines',name='Saw'))
fig.update_xaxes(range=[-np.pi,np.pi])
fig.update_yaxes(range=[-1,1])
N=100

x=np.linspace(-np.pi,np.pi,N)
k=np.linspace(0,N,N)
#print(fx)
a=0
b=0

@interact(a=(0.1,3,0.1),b=(-1.1, 1.1, 0.1))
def update(FourierRepresentation=False,a=0.1,b=0.1):
    with fig.batch_update():
        fig.data[0].y=saw(x,a,b)
        if(FourierRepresentation):
            if(len(fig.data)==1):
                #cn=SawFourierAnalytical(a,b,N)
                cn=saw(x,a,b,fourier=True,N=100)
                fx=FourierSeries(cn,x,N)
                plot(fig,x,fx,'FourierRepresentation')
            else:
                cn=saw(x,a,b,fourier=True,N=100)
                fx=FourierSeries(cn,x,N)
                fig.data[1].y=fx
        else:
            fig.data = [fig.data[0]]
        fig.show()
    



interactive(children=(Checkbox(value=False, description='FourierRepresentation'), FloatSlider(value=0.1, descr…

## 3. Exponential

In [7]:

fig=go.Figure()

fig.add_trace(go.Scatter(x=x,y=exp(x,0,0),mode='lines',name='Exp'))
#fig.update_xaxes(range=[-np.pi,np.pi])
#fig.update_yaxes(range=[-1,1])
N=100
M=80
x=np.linspace(-np.pi,np.pi,N)
k=np.linspace(0,N,N)
#print(fx)
a=0
b=0

@interact(a=(0.1,3,0.1),b=(-1.1, 1.1, 0.1),c=(-3,1,0.1))
def update(FourierRepresentation=False,a=0.1,b=0.1,c=0):
    with fig.batch_update():
        fig.data[0].y=exp(x,a,b,c)
        if(FourierRepresentation):
            if(len(fig.data)==1):
                #cn=ExpFourierAnalytical(a,b,c,N)
                #cn=((np.dot(fft(N,M),exp(x,a,b,c)))/N)
                cn=exp(x,a,b,c,fourier=True,N=100)
                fx=FourierSeries(cn,x,N)
                plot(fig,x,fx,'FourierRepresentation')
            else:
                #cn=ExpFourierAnalytical(a,b,c,N)
                #cn=((np.dot(fft(N,M),exp(x,a,b,c)))/N)
                cn=exp(x,a,b,c,fourier=True,N=100)
                fx=FourierSeries(cn,x,N)
                fig.data[1].y=fx
        else:
            fig.data = [fig.data[0]]
        fig.show()





interactive(children=(Checkbox(value=False, description='FourierRepresentation'), FloatSlider(value=0.1, descr…

## 4. Sinusoid

In [8]:

fig=go.Figure()

fig.add_trace(go.Scatter(x=x,y=sinu(x,0,0,0),mode='lines',name='Exp'))
#fig.update_xaxes(range=[-np.pi,np.pi])
#fig.update_yaxes(range=[-1,1])
N=100
M=80
x=np.linspace(-np.pi,np.pi,N)
k=np.linspace(0,N,N)
#print(fx)
a=0
b=0

@interact(a=(0.1,3,0.1),b=(-1.1, 1.1, 0.1),c=(-3,1,0.1))
def update(FourierRepresentation=False,a=0.1,b=0.1,c=0):
    with fig.batch_update():
        fig.data[0].y=sinu(x,a,b,c)
        if(FourierRepresentation):
            if(len(fig.data)==1):
                #cn=SinuFourierAnalytical(a,b,c,N)
                cn=sinu(x,a,b,c,fourier=True,N=100)
                fx=FourierSeries(cn,x,N)
                plot(fig,x,fx,'FourierRepresentation')
            else:
                #cn=SinuFourierAnalytical(a,b,c,N)
                cn=sinu(x,a,b,c,fourier=True,N=100)
                fx=FourierSeries(cn,x,N)
                fig.data[1].y=fx
        else:
            fig.data = [fig.data[0]]
        fig.show()




interactive(children=(Checkbox(value=False, description='FourierRepresentation'), FloatSlider(value=0.1, descr…

## 5. Gaussian

In [25]:

fig=go.Figure()

fig.add_trace(go.Scatter(x=x,y=gaus(x,0,0),mode='lines',name='Gaus'))
#fig.update_xaxes(range=[-np.pi,np.pi])
#fig.update_yaxes(range=[-1,1])
N=100
M=20
x=np.linspace(-np.pi,np.pi,N)
k=np.linspace(0,N,N)
#print(fx)
a=0
b=0

@interact(a=(0.1,5,0.1),b=(-1.1, 10, 0.1))
def update(FourierRepresentation=False,a=0.1,b=0.1):
    with fig.batch_update():
        fig.data[0].y=gaus(x,a,b)
        if(FourierRepresentation):
            if(len(fig.data)==1):
                #cn=((np.dot(fft(N,M),gaus(x,a,b)))/N)
                cn=gaus(x,a,b,fourier=True,N=N,M=M)
                
                fx=FourierSeries(cn,x,M)
                print(len(fx))
                plot(fig,x,fx,'FourierRepresentation')
            else:
                #cn=(np.dot(fft(N,80),gaus(x,a,b))/N)
                cn=gaus(x,a,b,fourier=True,N=N,M=M)
                fx=FourierSeries(cn,x,M)
                fig.data[1].y=fx
        else:
            fig.data = [fig.data[0]]
        fig.show()





interactive(children=(Checkbox(value=False, description='FourierRepresentation'), FloatSlider(value=0.1, descr…

In [10]:
fig=go.Figure()
a=2
b=3
orig=gaus(x,a,b)
cn=gaus(x,a,b,fourier=True,N=50,M=39)
fx=FourierSeries(cn,x,M)
plot(fig,x,orig,'Original Function')
plot(fig,x,fx,'FourierRepresentation')
fig

ValueError: shapes (39,50) and (200,) not aligned: 50 (dim 1) != 200 (dim 0)